# **Synchronize video files**

First we create the motif that we want to use for the synchronization process, in our case, the blink pattern of the LED.
The motif is generated as a kind of ground truth with a resolution of 1 ms (could be made adaptable, though). Upon loading
the motif template into a single cam data object, it will create all possible timeseries that could be observed, given the
framerate of the respective camera & the resolution of the template.

In [ ]:
%matplotlib widget

In [ ]:
from gait3d.sync_videos import MotifTemplate, MultiMotifTemplate

#no_blinking_template = MotifTemplate(led_on_time_in_ms=0, on_off_period_length_in_ms=100, motif_duration_in_ms=200)
regular_blinking_template = MotifTemplate(led_on_time_in_ms=50, on_off_period_length_in_ms=1_000, motif_duration_in_ms=4_000)
fast_blinking_template = MotifTemplate(led_on_time_in_ms=50, on_off_period_length_in_ms=100, motif_duration_in_ms=3_000)

In [ ]:
combined_template = MultiMotifTemplate()

In [ ]:
combined_template.add_motif_template(fast_blinking_template)
combined_template.add_motif_template(regular_blinking_template)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(combined_template.multi_motif_template)
plt.show()

Now that we have our template ready, let´s load the actual video file & construct our single cam data object!
Please note, since there is no DLC tracking data of the LED available as of yet, LED positions are hardcoded
and can be specified using the "led_marker_id" argument. Initializing the `cam_raw_calib_data` object might
take quite some time, as the LED pixel intensities have to be extracted from each & every frame of the video!

In [ ]:
from gait3d.sync_videos import SingleCamRawCalibrationData
from pathlib import Path

filepath_video = Path('/mnt/c/Users/dsege/Downloads/220825/220825/220825_Bottom_Charuco.mp4')
cam = SingleCamRawCalibrationData(filepath_video = filepath_video, cam_id = 'Bottom')

### Manual detection of the LED coordinates:

Until the DLC networks are trained & validated, the coordinates of the LED have to be determined manually & then passed as a tuple as (row index, column index), like:

> `cam.extract_led_pixel_intensities_as_timeseries(led_center_row_col_idxs = (row_idx, column_idx))`

As soon as the DLC predictions are available (and the corresponding code has been implemented), these can be used instead in the same method:

> `cam.extract_led_pixel_intensities_as_timeseries(filepath_tracking = filepath_dlc_output, led_marker_id = 'LED5')`

In [ ]:
from gait3d.utils import plot_single_frame_of_video

In [ ]:
plt.close()
plot_single_frame_of_video(filepath=filepath_video)

In [ ]:
cam.extract_led_pixel_intensities_as_timeseries(led_center_row_col_idxs=(446, 629))

In [ ]:
plt.close()
plt.plot(cam.led_timeseries)
plt.xlim(0, 400)
plt.show()

Another to-be-properly-implemented-step:

The fps are saved as 160 in the metadata of the video files. However, frame capturing was deliberately triggered only at 125 fps.
This either has to be updated alread in the video files (ideally?), or become an argument of the method / or become hardcoded.
For now, we simply do it here:

In [ ]:
cam.fps

In [ ]:
if cam.fps == 160:
    cam.fps = 125

Time to see whether we can find a match of our template! You can use the `start_time` and `end_time` arguments to specify 
the interval (in ms) in which the alignment should be checked. This might be usefull, as a pattern could occur multiple times
in a single session but maybe you want to align to a specific occurance of the motif, rather than to the best match, i.e. 
for synchronization maybe only in a 20 s window right of the start?

In the created plot, blue represents the LED pixel intensity timeseries & orange the fps adjusted template motif with the offset
that results in the best alignment score. The top graph shows the raw data values (i.e. pixel intensities or binary 0 / 1, 
respectively). The lower graph shows the z-score normalized data, which is also used for calculating the alignment score.

In [ ]:
cam.cam_id

In [ ]:
plt.close()
best_match_offset, best_match_start_idx = cam.find_best_match_of_template(template=combined_template, end_time = 20_000)

If the alignment looks good (crisp & near perfect alignment of vertical lines - amplitude is not relevant), you can proceed and adjust your video to this synchronization. Using `target_fps`, 
you also have the option to downsample the video accordingly

In [ ]:
cam.write_synchronized_and_fps_adjusted_calibration_video(start_frame_idx = best_match_start_idx, offset = best_match_offset, target_fps = 30, max_frame_count = 1_000)